In [1]:
from google.api_core import page_iterator
from google.cloud import storage
import os
import pandas as pd 
from google.cloud import bigquery

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [63]:
#next step is to add the extra columns

def bucket_metadata(bucket_name,project_id):
    """Prints out a bucket's metadata."""
  
    storage_client = storage.Client(project_id)
    bucket = storage_client.get_bucket(bucket_name)

    #print(f"ID: {bucket.id}")
    #print(f"Name: {bucket.name}")
    #print(f"Storage Class: {bucket.storage_class}")
    #print(f"Location: {bucket.location}")
    #print(f"Location Type: {bucket.location_type}")

    data                  = {}
    data['bucket_id']     = bucket.id
    data['bucket_name']          = bucket.name
    data['bucket_storage_class'] = bucket.storage_class
    data['bucket_location']      = bucket.location
    data['bucket_location_type'] = bucket.location_type
    #json_data = json.dumps(data)

    return data

def file_path_policy_respected_check(file_path):
    #function that checks whether the path policy of a certain bucket is respected or not
    #each path should be in the following format 
    path_chuncks = file_path.split('/')[2:-1]

    if len(path_chuncks) == 5 : policy_respected = True
    else : policy_respected = False

    return policy_respected

def _item_to_value(iterator, item):
    return item

def list_directories(bucket_name, prefix):

    if prefix and not prefix.endswith('/'):
        prefix += '/'

    extra_params = {
        "projection": "noAcl",
        "prefix": prefix,
        "delimiter": '/'
    }

    gcs = storage.Client()

    path = "/b/" + bucket_name + "/o"

    iterator = page_iterator.HTTPIterator(
        client=gcs,
        api_request=gcs._connection.api_request,
        path=path,
        items_key='prefixes',
        item_to_value=_item_to_value,
        extra_params=extra_params,
    )

    return [x for x in iterator]

In [90]:
#definitions of helpers functions

def file_path_pipeline_code_distinct_versions_check(input_path):

  bucket = input_path.split('/')[2]
  prefix  = input_path.split(bucket)[1][1:-4]

  #build a list containing path and corresponding sizes

  versions_sizes = list()

  for directory in list_directories(bucket,prefix):

    path = "gs://"+ bucket + "/" +  directory 
    print(path)
    size = os.popen("gsutil du -s " + str(path)).read().split()
    print("Size : " + str(size))

    versions_sizes.append(size)
  return versions_sizes


In [15]:
project_id = 'advanced-analytics-278408'
client = bigquery.Client(project=project_id)

query = '''
SELECT
*
FROM `advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params`
WHERE is_to_be_monitored = true
and landing_zone = 'GCS';
'''

df = client.query(query).to_dataframe()
df.columns

Index(['project_id', 'dataset_id', 'table_id', 'complete_table_id',
       'table_code', 'is_to_be_monitored', 'current_development_status',
       'current_development_status_id', 'source_system_id',
       'source_system_name', 'source_system_ownership_type', 'extraction_tool',
       'extraction_script', 'extraction_is_scheduled',
       'extraction_schedule_pattern', 'landing_zone', 'landing_zone_path',
       'landing_zone_form', 'loading_tool', 'load_script', 'load_is_scheduled',
       'load_schedule_pattern', 'load_form', 'is_dx_framework_applied',
       'id_field', 'data_loop_field', 'data_loop_granularity',
       'overall_documentation_URL', 'to_do_pending', 'notes'],
      dtype='object')

In [6]:
df.loc[0,'My_new_col'] = 'new_inserted_value'

In [56]:
query_2 = '''
SELECT
*
FROM `advanced-analytics-278408.user__rubini_riccardo.9898__pipelines_target_landing_zone_metadata`;
'''

df_9898 = client.query(query_2).to_dataframe()

In [57]:
df_9898.dtypes

analysis_date                                      object
analysis_date_time                                 object
project_id                                         object
dataset_id                                         object
table_id                                           object
complete_table_id                                  object
table_code                                         object
is_to_be_monitored                                 object
current_development_status                         object
current_development_status_id                      object
source_system_id                                   object
source_system_name                                 object
source_system_ownership_type                       object
extraction_tool                                    object
extraction_script                                  object
extraction_is_scheduled                            object
extraction_schedule_pattern                        object
landing_zone  

In [100]:
import datetime

#fill the first N column of df_9898 with the first N columns of 9891__monitoring_information_and_params
#define a list of columns to fill 

columns_list = [
      'project_id', 'dataset_id', 'table_id', 'complete_table_id',
       'table_code', 'is_to_be_monitored', 'current_development_status',
       'current_development_status_id', 'source_system_id',
       'source_system_name', 'source_system_ownership_type', 'extraction_tool',
       'extraction_script', 'extraction_is_scheduled',
       'extraction_schedule_pattern', 'landing_zone', 'landing_zone_path',
       'landing_zone_form', 'loading_tool', 'load_script', 'load_is_scheduled',
       'load_schedule_pattern', 'load_form'
]


#loop over the columns to fill 

for i in range(2):

  #print the filepath analised
  print(df.loc[i,'landing_zone_path'])

  #fill the date

  df_9898.loc[i,'analysis_date'] = datetime.datetime.now().strftime('%Y-%m-%d')
  df_9898.loc[i,'analysis_date_time'] = datetime.datetime.now()

  #add columns from the 9891
  for col in columns_list:

    df_9898.loc[i,col] = df.loc[i,col]

  bucket = df.loc[i,'landing_zone_path'].split('/')[2]
  bucket_meta = bucket_metadata(bucket,project_id)
  
  #add bucket metadata
  for key in bucket_meta:
    df_9898.loc[i,key] = bucket_meta[key] 

  #check that the policy and path info
  df_9898.loc[i,'file_path_policy_respected'] = file_path_policy_respected_check(df_9898.loc[i,'landing_zone_path'])
  df_9898.loc[i,'file_path_source_system'] = df_9898.loc[i,'landing_zone_path'].split('/')[5]
  df_9898.loc[i,'file_path_pipeline_code'] = df_9898.loc[i,'landing_zone_path'].split('/')[4]

  #extract bucket and prefix for a given landing_zone_path
  bucket = df_9898.loc[i,'landing_zone_path'].split('/')[2]
  prefix  = df_9898.loc[i,'landing_zone_path'].split(bucket)[1][1:-4]
  #count the number of v1 v2 v3 
  print(bucket)
  print(prefix)
  print(list_directories(bucket,prefix))

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions'] = len(list_directories(bucket,prefix))

  #compute the size of each versioning of each folder

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions_sizes'] = str(file_path_pipeline_code_distinct_versions_check(df_9898.loc[i,'landing_zone_path']))



gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3/
swarovski-advanced-analytics-data-ingestion-test
clean/00_SAP_P30/0004
['clean/00_SAP_P30/0004/v1/', 'clean/00_SAP_P30/0004/v2/', 'clean/00_SAP_P30/0004/v3/']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v1/
Size : ['552271279', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v1']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v2/
Size : ['84698956', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v2']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3/
Size : ['233904036', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0004/v3']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/
swarovski-advanced-analytics-data-ingestion-test
clean/00_SAP_P30/0006
['clean/00_SAP_P30/0006/v0/', 'clean/00_SAP_P30/0006/v1/', 'cl

In [101]:
file_path_pipeline_code_distinct_versions_check(df_9898.loc[i,'landing_zone_path'])
df_9898['file_path_pipeline_code_distinct_versions_sizes']

gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v0/
Size : ['160705', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v0']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v1/
Size : ['20401428471', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v1']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/
Size : ['16806465304', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2']


0    [['233904036', 'gs://swarovski-advanced-analyt...
1    [['16806465304', 'gs://swarovski-advanced-anal...
Name: file_path_pipeline_code_distinct_versions_sizes, dtype: object

In [99]:
bucket = df_9898.loc[i,'landing_zone_path'].split('/')[2]
prefix  = df_9898.loc[i,'landing_zone_path'].split(bucket)[1][1:-4]

#build a list containing path and corresponding sizes

versions_sizes = list()

for directory in list_directories(bucket,prefix):

  path = "gs://"+ bucket + "/" +  directory 
  print(path)
  size = os.popen("gsutil du -s " + str(path)).read().split()
  print("Size : " + str(size))

  versions_sizes.append(size)

print(str(versions_sizes))

gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v0/
Size : ['160705', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v0']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v1/
Size : ['20401428471', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v1']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/
Size : ['16806465304', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2']
[['160705', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v0'], ['20401428471', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v1'], ['16806465304', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2']]


In [86]:
bucket = df_9898.loc[i,'landing_zone_path'].split('/')[2]
prefix  = df_9898.loc[i,'landing_zone_path'].split(bucket)[1][1:-1]

for directory in list_directories(bucket,prefix):

  path = "gs://"+ bucket + "/" +  directory 
  print(path)
  size = os.popen("gsutil du -s " + str(path)).read().split()
  print("Size : " + str(size))


gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2018/
Size : ['68821689', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2018']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2019/
Size : ['54734759', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2019']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2020/
Size : ['5349025804', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2020']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2021/
Size : ['6028383092', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2021']
gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2022/
Size : ['5305499960', 'gs://swarovski-advanced-analytics-data-ingestion-test/clean/00_SAP_P30/0006/v2/2022']


In [102]:
dataset_id = 'user__rubini_riccardo'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = False
job_config.write_disposition = "WRITE_TRUNCATE"
#job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
load_job = client.load_table_from_dataframe(df_9898, dataset_ref.table("0001_dq_test"), job_config=job_config, location="EU")    # API request
print("Starting job {}".format(load_job))

/usr/local/lib/python3.8/dist-packages/google/cloud/bigquery/_pandas_helpers.py:552: UserWarning: Pyarrow could not determine the type of columns: bucket_created, file_path_pipeline_code_checked_version, file_number_in_landing_zone_path, file_size_in_landing_zone_path, file_oldest_creation_datetime, file_newest_creation_datetime, file_name_policy_respected, file_name_pipeline_code_info, file_name_pipeline_name_info, file_name_pipeline_windowframe_info, file_name_pipeline_history_min_info, file_name_pipeline_history_max_info, files_period_covered, files_period_missing_between_min_max, warning_flag_status.
  warnings.warn(


Starting job LoadJob<project=advanced-analytics-278408, location=EU, id=dc88373d-b51e-4899-acbc-dd96180e57ca>
